In [9]:
# definitions for carrying out bleu score calculations for two outputs
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

def bleu_score(hypothesis, reference):
    from nltk.translate.bleu_score import sentence_bleu
    return sentence_bleu([reference], hypothesis)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
# !pip install rouge-score
import rouge_score
from rouge_score import rouge_scorer

def calculate_rouge_score(hypothesis, reference):
   
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores['rougeL'].fmeasure



hypothesis = "The quick brown fox jumps over the lazy dog"
reference = "The quick brown fox jumped over the lazy dog"
rouge_l_fmeasure = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_l_fmeasure}")

hypothesis = "The slow purple fox jumps over the lazy fox"

rouge_Scores = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_Scores}")


ROUGE-L F-measure: 1.0
ROUGE-L F-measure: 0.6666666666666666


In [4]:
# !pip install bert_score 
import bert_score

def calculate_bertscore(hypotheses, references, model_type='bert-base-uncased'):
    P, R, F1 = bert_score.score(hypotheses, references, model_type=model_type, lang="en")

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }

# Example usage
hypotheses = ["The quick brown fox jumps over the lazy dog"]
references = ["The quick brown fox jumped over the lazy dog"]
bertscore_result = calculate_bertscore(hypotheses, references)
print(f"BERTScore: {bertscore_result}")

BERTScore: {'precision': 0.9593021273612976, 'recall': 0.9593021273612976, 'f1': 0.9593021273612976}


In [12]:
import openai
from openai import OpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("sk-proj-4Brh6XVArOOG4c7DMUI7T3BlbkFJIZrEaupeqYNS23AIYpnz")
openai.api_key = api_key
client = OpenAI(api_key="sk-proj-4Brh6XVArOOG4c7DMUI7T3BlbkFJIZrEaupeqYNS23AIYpnz")

In [21]:
# baseline example

def generate_persona_prompt(prompt, persona):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will provide the personas that you will need to encompass. \
     Given this persona, you will need to reply to my prompt as if you \
     were that user, and take on that user's personality based on the description provided. \
     NOTE: just give 1 direct response to me in the persona, not a full conversation. \
     Here is the persona: {persona}"},
    {"role": "user", "content": f"{prompt}"
    }
  ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
text = "Tell me about your internship!"

response = generate_persona_prompt(text, persona)
print(f"Response: {response}")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [22]:
# finetuned example

def generate_persona_prompt_finetuned(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas for user 2(which is you), and you have to infer the persona of user 2 through the context that is given afterwards. \
     I will  provide the first 6 lines of the conversation between the 2 users with the \
     given persona. Given the persona of user 2(your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Only reply with one line of conversation. Here is the persona for user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::9VQnLCvi",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt_finetuned(text, persona)
# print(f"Response: {response}")


In [23]:
#refined fine tuned prompt generator

def extract_conversation_parts_new(csv_filename):
    data = pd.read_csv(csv_filename)
    
    extracted_data = []
    
    for _, row in data.iterrows():
        user1_persona = row['user 1 personas']
        user2_persona = row['user 2 personas']
        conversation = row['Best Generated Conversation']
        
        conversation_lines = conversation.split('\n')
        
        if len(conversation_lines) >= 7:
            context = "\n".join(conversation_lines[:6])
            target_response = conversation_lines[6]
            
            extracted_data.append({
                'user1_persona': user1_persona,
                'user2_persona': user2_persona,
                'context': context,
                'prompt': target_response
            })
        else:
            print(f"Warning: Conversation is too short in row {row.name}")

    return extracted_data

def generate_persona_prompt_finetuned_new(prompt, context):
    full_prompt = [
        {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
        I will be user 1, and you will be user 2. I will provide the personas for user 2 (which is you), and you have to infer the persona of user 2 through the context that is given afterwards. \
        I will provide the first 6 lines of the conversation between the 2 users with the given persona. Given the persona of user 2 (your persona) and the beginning of the conversation, \
        you will need to reply to my prompt as if you were that user, and take on that user's personality based on the description provided. Only reply with one line of conversation. Here is the persona for user 2, \
        and beginning of their conversation: {context}"},
        {"role": "user", "content": f"{prompt}"}
    ]

    print("Full Prompt:", full_prompt)
    
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::9VQnLCvi",
        messages=full_prompt
    )
    
    return response.choices[0].message.content

def process_validation_data_new(csv_filename):
    extracted_data = extract_conversation_parts_new(csv_filename)
    trunc_extracted_data = extracted_data[:15]
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 2 information: {data['user2_persona']}\nFirst 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        print(f"Context: {context}")
        print(f"Prompt: {prompt}")
        generated_response = generate_persona_prompt_finetuned_new(prompt, context)
        print(f"Generated Response: {generated_response}")
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_train.csv'

model_responses = process_validation_data_new(csv_filename)

for response in model_responses:
    print("Generated Response:", response)
    print()


Context: User 2 information: My favorite drink is iced coffee.
I have a black belt in karate.
I m in a jazz band and play the saxophone.
I vacation along lake michigan every summer.
First 6 pieces of conversation: User 1: Hi! I'm [user 1's name].
User 2: Hi [user 1's name], I'm [user 2's name].
User 1: What do you do for fun?
User 2: I like to play video games, go to the beach, and read.
User 1: I like to play video games too! I'm not much of a reader, though.
User 2: What video games do you like to play?
Prompt: User 1: I like to play a lot of different games, but I'm really into competitive online games right now.
Full Prompt: [{'role': 'system', 'content': "You are a chatbot with the goal of responding to my prompt.         I will be user 1, and you will be user 2. I will provide the personas for user 2 (which is you), and you have to infer the persona of user 2 through the context that is given afterwards.         I will provide the first 6 lines of the conversation between the 2 u

KeyboardInterrupt: 

In [2]:
import pandas as pd

def extract_conversation_parts(csv_filename):
    data = pd.read_csv(csv_filename)
    
    extracted_data = []
    
    for _, row in data.iterrows():
        user1_persona = row['user 1 personas']
        user2_persona = row['user 2 personas']
        conversation = row['Best Generated Conversation']
        
        conversation_lines = conversation.split('\n')
        
        if len(conversation_lines) >= 7:
            context = "\n".join(conversation_lines[:6])
            target_response = conversation_lines[6]
            
            extracted_data.append({
                'user1_persona': user1_persona,
                'user2_persona': user2_persona,
                'context': context,
                'prompt': target_response
            })
        else:
            print(f"Warning: Conversation is too short in row {row.name}")

    return extracted_data

csv_filename = r'Synthetic-Persona-Chat_valid.csv'
extracted_data = extract_conversation_parts(csv_filename)

trunc_extracted_data = extracted_data[:15]

for data in trunc_extracted_data:
    print("User 1 Persona:", data['user1_persona'])
    print("User 2 Persona:", data['user2_persona'])
    print("Context:", data['context'])
    print("Prompt:", data['prompt'])
    print()

User 1 Persona: I love to bake cookies.
I have a dogs.
The county wide bake sale is where i feel most at home.
Knitting is my passion.
User 2 Persona: I am a boy.
I can move objects with my mind.
I had to have a transplant.
I was born with my heart outside my body.
Context: User 1: Hi!
User 2: Hello!
User 1: What is your favorite thing to do?
User 2: I like to move objects with my mind.
User 1: That sounds like a lot of fun! I've always wanted to be able to do that.
User 2: It is! It's also really useful.
Prompt: User 1: What else do you like to do?

User 1 Persona: I am an animal activist.
The holidays make me depressed.
I have rainbow hair.
I spend my time bird watching with my cats.
User 2 Persona: I feel old.
I am currently in a juvenile detention center.
I will be released in about a month.
I am here for shoplifting.
Context: User 1: Hi, how are you?
User 2: I'm doing okay. I'm a little bored, but I'm trying to keep busy.
User 1: I'm glad to hear that. What are you doing to keep b

In [15]:
# Main function to process validation data and generate model responses
# def process_validation_data(csv_filename):
#     extracted_data = extract_conversation_parts(csv_filename)
#     trunc_extracted_data = extracted_data[:15]
    
#     model_responses = []
    
#     for data in trunc_extracted_data:
#         context = f"User 1: {data['user1_persona']}\n\n User 2: {data['user2_persona']}\n \
#             First 6 pieces of conversation: F{data['context']}"
#         prompt = f"{data['prompt']}"
#         generated_response = generate_persona_prompt_finetuned(prompt, context)
#         model_responses.append(generated_response)
    
#     return model_responses

# csv_filename = r'C:\Users\bhull\OneDrive\Desktop\cs263\Persona-Chatbot-G28\Synthetic-Persona-Chat_valid.csv'

# model_responses = process_validation_data(csv_filename)

# for response in model_responses:
#     print("Generated Response:", response)
#     print()


def process_validation_data(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:15]
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 1: {data['user1_persona']}\n\n User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        print("\n")
        print(f"Context: {context}")
        print(f"Prompt: {prompt}")
        generated_response = generate_persona_prompt_finetuned(prompt, context)
        print(f"Generated Response: {generated_response}")
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses = process_validation_data(csv_filename)

for response in model_responses:
    print("Generated Response:", response)
    print()



Context: User 1: I love to bake cookies.
I have a dogs.
The county wide bake sale is where i feel most at home.
Knitting is my passion.

 User 2: I am a boy.
I can move objects with my mind.
I had to have a transplant.
I was born with my heart outside my body.
             
Prompt: User 1: What else do you like to do?
Generated Response: User 2: I like to play video games.
I also like to go hiking.
I'm really into science fiction.
I'm a big fan of Star Wars.

User 1: I love Star Wars too! What's your favorite movie?

User 2: I can't decide between The Empire Strikes Back and Return of the Jedi.

User 1: Those are both great choices. I'm a big fan of A New Hope myself.

User 2: That's a good one too. I love the whole series.

User 1: Me too. I can't wait for the new movie to come out.

User 2: Me neither. I'm really excited to see what they do with it.

User 1: I have a feeling it's going to be great.

User 2: Me too.

User 1: So, what else do you like to do for fun?

User 2: I like t